# Intro Dynamic World (Part 1)

[Tutorial site](https://developers.google.com/earth-engine/tutorials/community/introduction-to-dynamic-world-pt-1)

In [ ]:
import os

import ee
import folium
import toml
from folium import plugins
from IPython.display import Image
from pyprojroot import here

In [ ]:
service_email = toml.load(os.path.join(here(), ".git_ignore", "ee.toml"))["ee"][
    "SERVICE_EMAIL"
]

credentials = ee.ServiceAccountCredentials(
    service_email,
    os.path.join(here(), ".git_ignore", "ee-lsoa-land-use-02b30d7b328b.json"),
)
ee.Initialize(credentials)

We first define the variables containing the start date, end date, and coordinates of a location. Here we have defined a point centered at the Quabbin Reservoir in Massachusetts, US.

**The images in this collection have names that match the individual Sentinel-2 L1C product names from which they were derived.**

The first part of this tutorial is about comparing sentinel 2 imagery with dynamic world classes

In [ ]:
startDate = "2021-10-15"
endDate = "2021-10-25"
geometry = ee.Geometry.Point(-72.28525, 42.36103)

## Load a Sentinel-2 Image
We can find a Sentinel-2 image by applying filters on the Sentinel-2 L1C harmonized collection for the date range and location of interest. Since the Dynamic World classification is available only for scenes with < 35% cloud cover, we also apply a metadata filter.

In [ ]:
s2 = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterDate(startDate, endDate)
    .filterBounds(geometry)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 35))
)
s2

The resulting collection in the s2 variable contains all images matching the filters. We can call .first() to extract a single image (the earliest one matching our criteria) from the collection. Once we have the image, let's add it to the map to visualize it. The code also centers the viewport at the coordinates of the point location.

In [ ]:
s2_img = ee.Image(s2.first())
s2_img

In [ ]:
# Create a URL to the styled image
url = s2_img.getThumbUrl(
    {
        "min": 0,
        "max": 3000,
        "dimensions": 512,
        "bands": ["B4", "B3", "B2"],
    }
)

print(url)

# Display the thumbnail
print("\nPlease wait while the thumbnail loads, it may take a moment...")
Image(url=url)

***

## Find the Matching Dynamic World Image

To find the matching classified image in the Dynamic World collection, we need to extract the product id using the system:index property

In [ ]:
imageId = s2_img.get("system:index")
print(imageId)

In [ ]:
dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filter(
    ee.Filter.eq("system:index", imageId)
)
dwImage = ee.Image(dw.first())

In [ ]:
# Define the center of our map.
lat, lon = 42.36103, -72.28525
my_map = folium.Map(location=[lat, lon], zoom_start=12)
my_map

In [ ]:
# Handy func for adding image object as a folium layer


def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)


# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
classification = dwImage.select("label")
dwVisParams = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}

In [ ]:
# Set visualization parameters.
# Create a map.
lat, lon = 42.36103, -72.28525
my_map = folium.Map(location=[lat, lon], zoom_start=8)
# Add the land cover to the map object.
my_map.add_ee_layer(classification, vis_params=dwVisParams, name="DW Classification")
# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
# Display the map.
display(my_map)

In [ ]:
m = plugins.DualMap(location=(42.36103, -72.28525), tiles=None, zoom_start=8)
folium.TileLayer("cartodbpositron").add_to(m.m2)
folium.TileLayer("openstreetmap").add_to(m.m1)

fg_both = folium.FeatureGroup(name="markers_both")
icon_red = folium.Icon(color="red")
folium.Marker((42.36103, -72.28525), tooltip="both", icon=icon_red).add_to(fg_both)
fg_both.add_to(m)


m.m1.add_ee_layer(classification, vis_params=dwVisParams, name="DW Classification")

m.m2.add_ee_layer(
    s2_img,
    vis_params={"min": 0, "max": 3000, "bands": ["B4", "B3", "B2"]},
    name="LandSat Image",
)

m